# Homework 4: The Ising Model and the Metropolis–Hastings algorithm

The Ising Model has been introduced in the 20's by W. Lenz et E. Ising to explain the apparition of a spontaneous magnetization when we cool down a ferromagnetic material below a temperature called the Curie temperature. Here, we are supposing that we are placing quantum spins on a 2d square lattice that can take $\pm1$ value.

 <img src="images/ising_model.svg",width=300>

In a ferromagnetic medium, the spins tend to align therefore the interaction energy between 2 neighboring spins $i$ and $j$ is 

\begin{equation}
-Js_is_j
\end{equation}

where $J$ is a positive constant. The energy is minimized when the 2 spins have the same sign. If there is a magnetic field $h$, the interaction with the spin $i$ is

\begin{equation}
-hs_i.
\end{equation}

We call a microstate, values taken by the spins at a specific time. A macrostate is defined by macroscopic variables such as the temperature $T$ or the magnetization $M=\sum_i s_i$. The energy for a microstate is written as:

\begin{equation}
E = -J\sum_{(i,j)}s_is_j-h\sum_i s_i
\end{equation}
 
where $\sum_{(i,j)}$ is a sum taken on all the pairs of neighbors.

E. Ising has completely solved in 1925 the problem in 1 dimension (a linear chain of spin). The principal result is that this model does not yield any phase transition. In 1944, L. Onsager analytically solved the model in 2 dimensions on a square lattice in zero field. He proves the existence of a phase transition characterized by a critical temperature such that:

\begin{equation}
\sinh\frac{2J}{k_BT_c}=1
\end{equation}

## 1 Building the spin system

### 1.1 The initial square lattice of spins

We are going to write a function that initializes all the spins pointing up ($s_i=1$), identifies the position of each spin and marks the positions of their neighbors. First, We are going to establish the different steps related to this function. 

> Use the numpy function [`numpy.ones`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ones.html) to build a $N\times N$ matrix of ones and cast it into a pandas data frame. This pandas data frame should have as index and column names, integer values between 0 and $N-1$. This matrix represents the square lattice and $N^2$ is the total number of spins. Let's choose $N=50$ for this particular example.

In [ ]:
import pandas as pd
import numpy as np

N = 50

## TODO: create a pandas data frame with N columns and rows filled with 1s

initial_spins_df

### 1.2 The long format of the data

Because it is difficult to directly manipulate matrices in pandas are going to melt the matrix into a new dataframe such that one column will represent the x-position, one column will represent the y-position and the last column will represent the value of the spin

>- We need to reset the index of `initial_spins_df` to then melt it into a long format
- change the column names to "X", "Y" and "spin_value".
- reset the current index. This step is designed to keep a column that captures an unique index for each spin
- set a multiindex to be "X", "Y".

In [ ]:
# TODO: melt the initial_spins dataframe into a long dataframe

# TODO: Set the column names to "X", "Y" and "spin_value"

# TODO: reset the current index

# TODO: set the multiindex to be "X", "Y"

initial_spins_melted

### 1.3 The neighboring spins

We need now to establish the neighbors for each of the spins. In this particular case, we are going to consider a spin neighbor to be a spin that is directly or above, or below, or at the left or at the right of the spin we are considering 

<img src="images/ising_neighbors.jpg",width=400>

> Create 4 columns "up_spin", "down_spin", "left_spin" and "right_spin" that capture the value of the "index" column for the neighboring spins. For example, the index value of the spin at $X=0$ and $Y=0$ is 0 and the index value for its right neighbor ($X=1$ and $Y=0$) is 1. I suggest to use the `groupby` function on `level=0` or `level=1` and to apply the `shift` function with 1 or -1 as argument.

In [ ]:
# TODO: create the columns "up_spin", "down_spin", "left_spin" and "right_spin" with the index values
# of the neighboring spins

initial_spins_melted

### 1.4 Periodic boundary conditions

A thermodynamic system has of the order of $\sim10^{23}$ elements (spins here). To minimize the boundary effect of the small spin system we are constructing, we are going to assume that a spin at the extreme left of the lattice is connected on the left to a spin at the extreme right of the lattice with similar boundary conditions for the top and bottom spins of the lattice. This effectively warps the lattice onto a torus geometry

<img src="images/boundaries.png",width=400>

> - You may have noticed that in creating the neighboring spins columns, few elements of those columns remained `Nan`. Fill those missing values with index values of the spins such that the periodic boundary conditions are satisfied. For example, the spin at $X=0$ and $Y=0$ is on the top boundary and on the left boundary. We would find its up neighbor at $X=0$ and $Y=49$ and its left neighbor at $X=49$ and $Y=0$. This problem becomes easy by using the multiindex. 
- Ensure that the data type of "up_spin", "down_spin", "left_spin" and "right_spin" is integer by using the `astype` function.
- Reset the index and set as index the column "index".

The resulting data frame should look like

<img src="images/resulting_df.png",width=600>

In [ ]:
# TODO: replace the missing values in "up_spin", "down_spin", "left_spin" and "right_spin" 
# such that the periodic boundary counditions are satisfied

initial_spins_melted

### 1.5 To summarize

Let's summarize those steps into a function.

> Write a function that take as argument an integer $N$ and returns an initialized spin lattice into a long format with the neighboring spins and the period boundary conditions. It should returns exactly what was shown in the last cell.  

In [ ]:
## TODO: rewrite the past steps into the following function

def initialize_spin_lattice(N=50):
    ## YOUR CODE
    raise NotImplementedError
    
    
initialize_spin_lattice(N=50)

## 2 The Monte Carlo / Metropolis Simulation

In equilibrium statitical physics, the probability of a microstate $i$ (a specific spin configuration) is given by the Maxwell-Boltzmann distribution

\begin{equation}
p_i=\frac{e^{-\frac{E_i}{k_bT}}}{\sum_{i\in\mathcal{S}}e^{-\frac{E_i}{k_bT}}}
\end{equation}

where $E_i$ is the microstate energy, $k_b$ is the Boltzmann constant, $T$ is the temperature and $\mathcal{S}$ is the set of all possible microstates. $\sum_{i\in\mathcal{S}}e^{-\frac{E_i}{k_bT}}$ is simply a normalization constant. The expected value of an observable $A$ can be written as

\begin{equation}
\langle A\rangle=\frac{\sum_{i\in\mathcal{S}}A_ie^{-\frac{E_i}{k_bT}}}{\sum_{i\in\mathcal{S}}e^{-\frac{E_i}{k_bT}}}
\end{equation}

So given a system that has a discrete number of states, we could, using a computer, calculate A for each state and weight these values by their Boltzman factors to find the average $A$. This might be feasible for a system with a small number of states, but if we have a $20\times20$ spin lattice interacting via the Ising model, there are $2^{400}$ states, so we cannot possibly examine all of them.

Instead of sampling (measuring parameters like $A$ for) a lot of states and then weighting them by their Boltzman factors, it makes more sense to choose states based on their Boltzman factors and to then weight them equally. This is known as the Metropolis algorithm (a particular Monte Carlo method), which is an importance sampling technique. One pass through the algorithm is described here:

- A spin flip is made by randomly choosing one spin.
- The energy difference of the resulting state relative to the previous state, $\Delta E$, is calculated.
- If $\Delta E\leq 0$, the new state is energetically favorable and thus accepted. Otherwise, a random number $0\leq\eta\leq 1$ is generated, and the new state is only accepted if $\exp(-\Delta E/k_bT) > \eta$.

### 2.1 Computing $\Delta E$

Let's consider a spin $s_i$ having a value $\pm1$. The energy of a microstate is 

\begin{equation}
E = -J\sum_{(i,j)}s_is_j-h\sum_i s_i
\end{equation}

>What is the change in energy $\Delta E$ of a microstate if we flip that spin ($s_i \leftarrow s_i\times(-1)$)? 

\#\# TODO: compute analytically $\Delta E$ as a function of $J$, $s_i$, $h$ and $\sum_{j}s_j$ (the spin sum over the neighbors)

### 2.2 One Monte Carlo step

We now need to write the code for one Monte Carlo step (one spin flip). Actually because we are using a Pandas data frame, we can select many spins at once, flip them and test if we want to keep the new configurations or not. Because we just showed that the computation of  $\Delta E$ is local to a specific spin, we can only do this if we select spins that are not neighbors from each others. 

From this point forward, we are going to choose $k_b=J=1$ for simplicity. Here a pseudo-code to help you understand what need to be done for the following function.

```
def one_step(arguments -> the temperature T, the magnetic field h)

    sample_spins <- select few non neighboring spins at random

    for each spin in sample_spins:

        compute delta_e using h, the spin and its neighbors' values

        if delta_e <= 0 then:
            flip the spin
        else:
            flip the spin if exp(- delta_e / T) > x where x a uniform random number between 0 and 1  
```
    
Obviously all of this can be written with vectorize code (if not your code if going to be very slow!).

> Write the `one_step` function. Note that it does not return anything, it just decides to flip a bunch of spins of not.

In [ ]:
## TODO: write the function one_step 

def one_step(df,       # the dataframe to update
             T,        # the temperature 
             h,        # the magnetic field
             frac,     # the fraction of spins from the lattice to choose
             parityX,  # parameter to avoid to choose neighboring spins in the same batch
             parityY   # parameter to avoid to choose neighboring spins in the same batch
            ):
    
    # This part of the code is to ensure that spins chosen cannot be neighbors
    sample_spins = df.loc[(df["X"] % 2 == parityX) & (df["Y"] % 2 == parityY)]
    # we sample the resulting data frame
    sample_spins = sample_spins.sample(frac=frac)  
    
    ## TODO: compute delta_e for all the spins in sample_spins. delta_e is a vector
    
    ## TODO: Choose to flip the spins or not based on delta_e, T, and uniform random numbers
               
    ## No need to return anything            

### 2.3 Thermodynamic equilibrium


We need many Monte Carlo steps to reach thermodynamic (or statistical) equilibrium. This mean we need many steps to be sure that each spin was able to "feel" the temperature and magnetic field. I will not ask to write this part. I found the following values to be sufficient to reach equilibrium. I invite you of course to validate those values and test that the thermodynamic quantities have indeed reached an equilibrium. Here, as an example, I show the magnetization reaching thermodynamic equilibrium for a $50\times50$ spin lattice 

<img src="images/equilibrium.png",width=600>

In [ ]:
## The function to reach thermodynamic equilibrium for each value of T and h

def get_to_equilibrium(df,# the dataframe to update
                       T,# the temperature 
                       h,# the magnetic field
                       frac=0.8,# the fraction of spins from the lattice to choose
                       iteration=500 # the number of iteration of the one_step function
                      ):
    for i in range(iteration):
        
        # we choose at random 0 or 1 to select a sublattice with no neighboring spins
        parity = np.random.choice([0,1], size=2)
        
        # we apply the one_step function
        one_step(df, T, h, frac, parity[0], parity[1]) 

## 3  The Ising Model physics

### 3.1 The $h=0$ case 

Now that we have established how to initialize the spin lattice and that we can get those spins to reach a thermodynamic equilibrium for any temperature and magnetic field, we can start some experiments. We have going to vary the temperature and keep $h=0$ to capture the different thermodynamic quantities.

> - Choose different values of $T$ between 0 and 5 (I advise a finner grid between 1.75 and 2.5)
- Initialize a spin lattice into the `spin_lattice` dataframe. You can try different values of $N$. The greater $N$ is the better are going to be the results but there are going to take a greater computational time. $N=20$ is a good start to check that everything works correctly.
- For each temperature (for-loop is ok here), capture the state of the lattice into a dataframe `spin_evolution`. I advise to build this data frame such that it has as columns name the temperatures and as index, the index of the `spin_lattice`.

I personally obtain the following data frame for `spin_evolution`

<img src="images/spin_evolution.png",width=800>

Note that here, we are only capturing one microstate per temperature. To be true to statistical mechanics, we should in principle sample many times the systems for each temperature to make accurate measurements of the different thermodynamic quantities. For simplicity, I only ask for one sample but know that the quantities you will measure will then be subject to statistical fluctuation. You can try to reproduce the experiment many times to perform accurate averaged measurements.

In [ ]:
## TODO: create some temperatures. The following is just an example to refine!!
# temperatures = np.arange(0, 5.5, 0.5).round(decimals=4) 

## TODO: initialize the spin lattice
spin_lattice = ## YOUR CODE
spin_evolution = pd.DataFrame(index=spin_lattice.index, columns=temperatures)

## TODO: loop through the temperatures to capture the state of the spins system

If you have followed the same formatting that I advised, you can see the evolution of the spin lattice in the following animation. If it does not work, try to run it a few times. This specific library tend to not work 100% of the time and I invite you to modify the code to make it work with the data you have.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig = plt.figure(figsize=(10, 10))


sign = 1
i = len(temperatures) - 1 

spin_lattice["spin_value"] = spin_evolution[temperatures[i]]

im = plt.imshow(spin_lattice.pivot(index="X", columns="Y", values="spin_value").values, animated=True)
plt.xlabel("X", fontsize=20)
plt.ylabel("Y", fontsize=20)
text = plt.text(0, -3, str(temperatures[i]), fontsize=30)

def updatefig(*args):
    global spin_evolution, spin_lattice, temperatures, sign, i
    
    if (i == 0) or (i == len(temperatures) - 1):
         sign *= -1  
    i += 1 * sign

    spin_lattice["spin_value"] = spin_evolution[temperatures[i]]
    data = spin_lattice.pivot(index="X", columns="Y", values="spin_value")
    im.set_array(data)
    text.set_text("T = " + str(temperatures[i]))

    return im,

ani = animation.FuncAnimation(fig, updatefig, frames=100, interval=50, blit=True)


### 3.2 The thermodynamic quantities

We are going to focus on estimating from the Ising system, the energy per spin, the magnetization per spin, the specific heat and the magnetic susceptibility. The simplest quantity to estimate is the magnetization that is the average value of the spin value over the lattice:

\begin{equation}
M = \overline{s} = \frac{1}{N^2}\sum_{i=1}^{N^2}s_i
\end{equation}

I am using $\overline{s}$ to talk about the average over the lattice instead of $\langle s\rangle$ that would represent an ensemble average. The energy per spin can be found using

\begin{equation}
E = -J\sum_{(i,j)}s_is_j-h\sum_i s_i
\end{equation}
 
which per spin becomes for $J=1$ and $h=0$

\begin{equation}
\epsilon = -\overline{\frac{s}{2}\sum_{j}s_j}
\end{equation}

the factor 2 comes to avoid to sum twice the same spin pair. The specific heat can be written as a function of the variance of the energy. We call $\beta=1/k_bT$ (don't forget, we choose $k_b=1$) and $Z$ is the canonical partition function

\begin{eqnarray}
C_v &=& \frac{\partial \langle E\rangle}{\partial T} \\
&=& -\frac{\beta}{T}\frac{\partial \langle E\rangle}{\partial \beta} \\
&=& -\frac{\beta}{T}\frac{\partial^2 \ln Z}{\partial \beta^2} \\
&=& -\frac{\beta}{T}\frac{\partial}{\partial \beta}\left(\frac{1}{Z}\frac{\partial Z}{\partial \beta}\right) \\
&=& -\frac{\beta}{T}\left[\frac{1}{Z}\frac{\partial^2Z}{\partial\beta^2}-\frac{1}{Z^2}\left(\frac{\partial Z}{\partial\beta}\right)^2\right] \\
&=& -\frac{\beta}{T}\left[\langle E^2\rangle - \langle E\rangle^2\right]
\end{eqnarray}

This the ensemble variance of the energy. You can estimate this variance per spin as we did for the average of the energy. In a similar manner, the magnetic susceptibility can be estimated from the magnetization

\begin{eqnarray}
\chi &=& \frac{\partial \langle M\rangle}{\partial h} \\
&=& \beta\left[\langle M^2\rangle - \langle M\rangle^2\right]
\end{eqnarray}

> Parse those equation to write functions that estimate those quantities onto our spin lattice. Use the `apply` function to iterate though the `spin_evolution` data frame columns to compute those quantities for all temperatures. You can use the attribute `name` (`x.name`) to access the name of a column (here it is going to be the temperature) within the `apply` function.  

In [ ]:
## TODO: write a function to compute the average energy of the spin system
def compute_energy_mean(x, df):
    ## YOUR CODE
    raise NotImplementedError

## TODO: write a function to compute the magnetization of the spin system
def compute_magnetization(x):
    ## YOUR CODE
    raise NotImplementedError

## TODO: write a function to compute the specific heat of the spin system
def compute_specific_heat(x, df):
    ## YOUR CODE
    raise NotImplementedError

## TODO: write a function to compute the magnetic susceptibility of the spin system
def compute_magnetic_susceptibility(x):
    ## YOUR CODE
    raise NotImplementedError

statistics_df = pd.DataFrame(columns=["Energy", "Magnetization", "Specific_heat", "Magnetic_susceptibility"],
                            index=temperatures)

statistics_df["Energy"] = spin_evolution.apply(compute_energy_mean, args=(spin_lattice,))
statistics_df["Magnetization"] = spin_evolution.apply(compute_magnetization)
statistics_df["Specific_heat"] = spin_evolution.apply(compute_specific_heat, args=(spin_lattice,))
statistics_df["Magnetic_susceptibility"] = spin_evolution.apply(compute_magnetic_susceptibility)

statistics_df

> Let's now plot the resulting curves as a function of the temperature. You can use this [paper](http://216.92.172.113/courses/phys39/simulations/Student%20Ising%20Swarthmore.pdf) to get a better idea of what you should see. The ising model has a critical temperature (the temperature where the phase transition happens) of $T_c=\frac{2}{\log(1+\sqrt{2})}$. Add using the matplotlib function `axvline`, a vertical red line for $T=T_c$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
Tc = 2./ np.log(1. + np.sqrt(2))

## TODO: plot the energy per spin as a function of the temperature
## TODO: add a vertical red line at T = Tc

In [ ]:
## TODO: plot the magnetization per spin as a function of the temperature
## TODO: add a vertical red line at T = Tc

In [ ]:
## TODO: plot the Specific heat per spin as a function of the temperature
## TODO: add a vertical red line at T = Tc

In [ ]:
## TODO: plot the Magnetic susceptibility per spin as a function of the temperature
## TODO: add a vertical red line at T = Tc

### 3.3 Few comments

- The energy should increase $\propto k_bT$ for large temperature. If you plot $\epsilon/T$ vs $T$, the curve should become flat and reach a constant.
- You see the magnetization becoming null for a specific value of the temperature
- You see the Specific heat and the Magnetic susceptibility reaching a sharp peak for a specific value of the temperature
- If $N\rightarrow \infty$, the peaks should agree with the exact theoretical value of $T_c$.

According to the conventional classification of phase transitions, a transition is first-order if the energy is discontinuous with respect to the order parameter (i.e., in this case, the temperature), and second-order if the energy is continuous, but its first derivative with respect to the order parameter is discontinuous, etc. We conclude that the loss of spontaneous magnetization in a ferromagnetic material as the temperature exceeds the critical temperature is a second-order phase transition.


### 3.4 The critical exponents

The concept of [critical exponents](https://en.wikipedia.org/wiki/Critical_exponent) also characterizes a phase transition. Quite generally, as a critical point (continuous phase transition) is approached from below, the different thermodynamic quantities behave as power-law functions of the temperature.

The magnetization
\begin{equation}
\langle M \rangle \sim (T_c-T)^\beta
\end{equation}

The Magnetic susceptibility
\begin{equation}
\chi \sim (T_c-T)^{-\gamma}
\end{equation}

The Specific heat
\begin{equation}
C_v \sim (T_c-T)^{-\alpha}
\end{equation}

The exact theoretical values for those exponents are $\alpha=0$, $\beta=1/8$ and $\gamma = 7/4$ for the 2D Ising model.

>Remember that if 
\begin{equation}
y = ax^b
\end{equation}
then
\begin{equation}
\log(y) = \log(a) + b\log(x)
\end{equation}
Use this fact to perform a linear regression between $\log(T_c-T)$ and $\log(M)$ for few points below the critical temperature. You can use the `linregress` function from the `scipy.stats` package. Reproduce this process with $\chi$ and $C_v$. With this linear regression, you should be able to estimate the critical exponent $\beta$, $\alpha$ and $\gamma$.

You could visual the quality of the fit by plotting the results in a log-log plot. I personally obtain something looking like
<img src="images/magn.png",width=600>

In [ ]:
## TODO: estimate beta by perfoming a linear regression

In [ ]:
## TODO: estimate alpha by perfoming a linear regression

In [ ]:
## TODO: estimate gamma by perfoming a linear regression

### 3.5 The $h\neq0$ case 

>We are now look at the general case where $h\neq0$. Create a vector of magnetic field between -2 and 2 with small enough incremental steps. We first going to look at the case $T>T_c$. Choose a temperature $T>T_c$ ($T=3$ for example) and capture the evolution of the spins by iterating the magnetic fields from -2 to 2 and then iterating backward, from 2 to -2. Plot the magnetization for the forward pass on the same chart than the backward pass.

In [ ]:
## TODO: Iterate forward and backward the magnetic fields for T > T_c
## TODO: plot the resulting evolution of the magnetization

>Now repeat the same experiment but choose a temperature $T<T_c$ (1 for example) and plot the resulting magnetization for the backward and forward pass on the same chart.

In [ ]:
## TODO: Iterate forward and backward the magnetic fields for T < T_c
## TODO: plot the resulting evolution of the magnetization

You should observe that the magnetization is subject to a Hysteresis Loop for $T<T_c$ characteristic of the ferromagnetic systems.